# 1. Crawler

Murashko Artem SD20-01 | ar.murashko@innopolis.university

## Imports

In [2]:
import argparse
import os
import re
import requests
import hashlib
import requests
import urllib.parse

from bs4 import BeautifulSoup
from bs4.element import Comment
from collections import Counter
from queue import Queue
from nltk.tokenize import word_tokenize
from string import punctuation

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [3]:
def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='download file.')
    parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
    parser.add_argument("url", type=str, default=None, help="Provide URL here")
    args = parser.parse_args()
    wget(args.url, args.filename)

usage: ipykernel_launcher.py [-h] [-O FILENAME] url
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [5]:
def get_hash_filename(name):
    hash_filename = hashlib.md5(name.encode("utf-8")).hexdigest()
    return hash_filename


class Document:

    def __init__(self, url):
        self.content = None
        self.url = url
        
        # Try to create root folder for downloaded files
        self.folder_name = "urls_data/"
        try:
            if not os.path.exists(self.folder_name):
                os.makedirs(self.folder_name)
        except:
            # In case of error store in the root folder of the project
            self.folder_name = ""

    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()

    def download(self):
        try:
            print(f"Trying to download contents of {self.url}")
            resp = requests.get(self.url, allow_redirects=True, timeout=10)
        except requests.exceptions.Timeout:
            print(f"Timeout for getting from {self.url} exceeded.")
            return False
        except requests.exceptions.TooManyRedirects:
            print(f"Too many redirects for {self.url}.")
            return False
        except Exception as e:
            print(f"Error requesting from {self.url}:", e)
            return False

        self.content = resp.content
        return resp.status_code == requests.codes.ok

    def persist(self):
        hash_filename = get_hash_filename(self.url)

        try:
            with open(self.folder_name + hash_filename, 'wb') as f:
                f.write(self.content)
        except OSError:
            print(f"OS error writing {self.url} with hash {hash_filename}.")
            return False
        except Exception as e:
            print(f"Unexpected error writing {self.url} with hash {hash_filename}:", e)
            return False

    def load(self):
        hash_filename = get_hash_filename(self.url)

        try:
            with open(self.folder_name + hash_filename, 'rb') as f:
                self.content = f.read()
        except FileNotFoundError:
            print(f"File {self.url} with hash {hash_filename} not found.")
            print(f"Download it first.")
            return False
        except OSError:
            print(f"OS error opening {self.url} with hash {hash_filename}.")
            return False
        except Exception as e:
            print(f"Unexpected error opening {self.url} with hash {hash_filename}:", e)
            return False

        return True

### 1.1.1. Tests ###

In [6]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

File http://sprotasov.ru/data/iu.txt with hash 51f3175e65788346991dfedb4f6c9923 not found.
Download it first.
Trying to download contents of http://sprotasov.ru/data/iu.txt


## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [7]:
class HtmlDocument(Document):

    def __init__(self, url):
        super().__init__(url)

        self.anchors = []
        self.images = []
        self.text = ""
        self.text_list = []
    
    def parse(self):
        # soup
        soup = BeautifulSoup(self.content, 'html.parser')
        # In rare cases (usually when a UTF-8 document contains text written in a completely different encoding), 
        # the only way to get Unicode may be to replace some characters with the special Unicode 
        # character “REPLACEMENT CHARACTER” (U+FFFD, �).
        # We don't need to process such files, thus just skip parsing stage (a.k.a empty website)
        if soup.contains_replacement_characters:
            return

        # links
        self._parse_links(soup)

        # images
        self._parse_images(soup)

        # text
        self._parse_visible_text(soup)

    def _parse_links(self, soup):
        url_tags = soup.findAll('a')

        for line in url_tags:
            link = urllib.parse.urljoin(self.url, line.get('href'))
            text = line.text
            self.anchors.append((text, link))

    def _parse_images(self, soup):
        img_tags = soup.findAll('img')

        for line in img_tags:
            link = urllib.parse.urljoin(self.url, line.get('src'))
            self.images.append(link)

    def _parse_visible_text(self, soup):

        def is_visible_tag(element):
            # checking for not needed tags
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            
            # checking for comments
            if isinstance(element, Comment):
                return False
            
            # checking for empty string
            if not element or not element.strip():
                return False

            return True

        texts = soup.findAll(text=True)
        visible_texts = filter(is_visible_tag, texts)
        
        for t in visible_texts:
            self.text_list.append(t.strip()) # leave as is
            self.text += t.strip()  # join in one string

### 1.2.1. Tests

In [8]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

File http://sprotasov.ru with hash a5acd174e703c50ece4121fd4a676468 not found.
Download it first.
Trying to download contents of http://sprotasov.ru


## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria to succeed in the task**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [9]:
class HtmlDocumentTextData:

    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()

    def get_sentences(self):
        result = []
        temp = [t.split('.') for t in self.doc.text_list]
        for t in temp:
            for sentence in t:
                sen = sentence.strip()
                if sen == "":
                    continue

                result.append(sen)

        return result

    def get_word_stats(self):
        cnter = Counter()
        for sentence in self.get_sentences():
            # Here we can use re.findall(r'\w+', sentence) for words parsing,
            # but nltk library do it better with the same performance.
            words = word_tokenize(sentence)
            for word in words:
                if word in punctuation + '«' + '»':
                    continue

                cnter[word.lower()] += 1

        return cnter

### 1.3.1. Tests ###

In [10]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

File https://innopolis.university/ with hash 9789081d4f18a3ad9d95593dcb27974b not found.
Download it first.
Trying to download contents of https://innopolis.university/
[('и', 46), ('в', 23), ('иннополис', 22), ('на', 14), ('с', 14), ('университет', 12), ('университета', 12), ('по', 10), ('центр', 10), ('для', 9)]


## 1.4. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [11]:
class Crawler:

    def crawl_generator(self, source, depth=1):
        # TODO return real crawling results. Don't forget to process failures,
        # exceptions, 3**, 4** codes
        if depth <= 0:
            return

        try:
            text_data = HtmlDocumentTextData(source)
        except Exception as e:
            print(f"Cannot get links from root source: {e}")
            return

        # Passing the root document
        yield text_data

        # Used to keep track of visited links.
        # If we've already visited current, do not do this again.
        visited_links = {source: True}

        # Initing the queue and add all the links on the zero depth
        queue = Queue()
        for _, link in text_data.doc.anchors:
            if visited_links.get(link) is not None:
                continue

            queue.put(link)
            visited_links[link] = True

        # Number of links left on the current depth
        links_on_depth = queue.qsize()
        curr_depth = 1

        # Breadth First Search algorithm
        while not queue.empty() and curr_depth <= depth:
            # If there are no more links on the current depth left,
            # increase current depth and continue BFS.
            if links_on_depth == 0:
                links_on_depth = queue.qsize()
                curr_depth += 1
                continue
            
            print(f"{links_on_depth} links left on depth {curr_depth}/{depth}")
            
            # Get a link and decrease the number of links on the current
            # depth left.
            link = queue.get()
            links_on_depth -= 1

            # Try to fetch data from the link
            try:
                curr_text_data = HtmlDocumentTextData(link)
            except:
                continue

            # Passing the current document
            yield curr_text_data

            # Add all anchors to the queuee from the current document
            for _, link in curr_text_data.doc.anchors:
                if visited_links.get(link) is not None:
                    continue

                queue.put(link)
                visited_links[link] = True

### 1.4.1. Tests ###

In [12]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

File https://innopolis.university/en/ with hash 2aac722811443b24ae4282bf5f6bbd14 not found.
Download it first.
Trying to download contents of https://innopolis.university/en/
https://innopolis.university/en/
346 distinct word(s) so far
64 links left on depth 1/2
https://innopolis.university/
880 distinct word(s) so far
63 links left on depth 1/2
File https://apply.innopolis.university/en with hash fda4ea7e2c007607ef46f063b70ffaa7 not found.
Download it first.
Trying to download contents of https://apply.innopolis.university/en
https://apply.innopolis.university/en
1517 distinct word(s) so far
62 links left on depth 1/2
File https://innopolis.university/proekty/activity/en" with hash 5387394fd5cbb69d22e28f30075f484c not found.
Download it first.
Trying to download contents of https://innopolis.university/proekty/activity/en"
61 links left on depth 1/2
File https://media.innopolis.university/en with hash e3d10e1639f6e81aa12286fba3247def not found.
Download it first.
Trying to download co

https://innopolis.university/en/team-structure/team-faculty/
4959 distinct word(s) so far
35 links left on depth 1/2
File https://innopolis.university/en/team-structure/team-faculty2/ with hash 95c1df6976338c2f598cec4c065218a6 not found.
Download it first.
Trying to download contents of https://innopolis.university/en/team-structure/team-faculty2/
https://innopolis.university/en/team-structure/team-faculty2/
4968 distinct word(s) so far
34 links left on depth 1/2
File https://innopolis.university/en/nir2022/ with hash c353ad75199e99e395d2f16174cbb526 not found.
Download it first.
Trying to download contents of https://innopolis.university/en/nir2022/
https://innopolis.university/en/nir2022/
5130 distinct word(s) so far
33 links left on depth 1/2
File https://innopolis.university/en/proekty/activity/ with hash cdeb1b5426dc0b8737caa1d9b075da4b not found.
Download it first.
Trying to download contents of https://innopolis.university/en/proekty/activity/
https://innopolis.university/en/pro

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
11 links left on depth 1/2
File https://t.me/universityinnopolis with hash 8f2c0b3806287bc5c702c6ae19612485 not found.
Download it first.
Trying to download contents of https://t.me/universityinnopolis
https://t.me/universityinnopolis
6166 distinct word(s) so far
10 links left on depth 1/2
File https://vk.com/innopolisu with hash 7b996b87662798515f1833bbdac4b04f not found.
Download it first.
Trying to download contents of https://vk.com/innopolisu
https://vk.com/innopolisu
6351 distinct word(s) so far
9 links left on depth 1/2
File https://www.youtube.com/user/InnopolisU with hash f40a355d92d5937bd4967cc6ca7a1674 not found.
Download it first.
Trying to download contents of https://www.youtube.com/user/InnopolisU
https://www.youtube.com/user/InnopolisU
6364 distinct word(s) so far
8 links left on depth 1/2
File

https://apply.innopolis.university/open-day/
11680 distinct word(s) so far
870 links left on depth 2/2
File https://apply.innopolis.university/grant/ with hash c8a8fecbbc8d0da413c635c21d29372b not found.
Download it first.
Trying to download contents of https://apply.innopolis.university/grant/
https://apply.innopolis.university/grant/
11776 distinct word(s) so far
869 links left on depth 2/2
File https://apply.innopolis.university/olympiad-bonus/ with hash b365f173ef83df9881a0fda9537d2ada not found.
Download it first.
Trying to download contents of https://apply.innopolis.university/olympiad-bonus/
https://apply.innopolis.university/olympiad-bonus/
11849 distinct word(s) so far
868 links left on depth 2/2
File https://apply.innopolis.university/faq/ with hash d01a9c4a78257ffe798d0209ae37f310 not found.
Download it first.
Trying to download contents of https://apply.innopolis.university/faq/
https://apply.innopolis.university/faq/
11948 distinct word(s) so far
867 links left on depth 2

...

https://innopolis.university/public/files/pps/PPS%202022.07.12.pdf
Skipping https://innopolis.university/public/files/pps/PPS%202022.07.12.pdf
2 links left on depth 2/2
File https://innopolis.university/public/files/pps/PPS%20SPIS.pdf with hash ebb597b2f02c57277aaf92701569e5c8 not found.
Download it first.
Trying to download contents of https://innopolis.university/public/files/pps/PPS%20SPIS.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://innopolis.university/public/files/pps/PPS%20SPIS.pdf
Skipping https://innopolis.university/public/files/pps/PPS%20SPIS.pdf
1 links left on depth 2/2
File https://career.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf with hash 221299664ea27ca435a0912276aba00e not found.
Download it first.
Trying to download contents of https://career.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://career.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf
Skipping https://career.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf
Done
[('и', 17540), ('the', 13856), ('в', 13353), ('and', 12306), ('of', 11306), ('in', 7228), ('to', 6687), ('на', 5854), ('university', 5126), ('по', 4837), ('с', 4686), ('a', 4292), ('for', 4255), ('иннополис', 3834), ('innopolis', 3683), ('я', 3091), ('для', 2732), ('i', 2664), ('университета', 2630), ('at', 2432)]
